In [1]:
#importujemy potrzebne biblioteki

import pandas as pd
import numpy as np
import pandas_profiling

In [2]:
#ustawiamy widoczność większej ilości kolumn

pd.set_option('display.max_columns',50)

In [3]:
df=pd.read_excel('default of credit card clients.xls',header=1,index_col='ID')

In [4]:
#ustawaimy część zmiennych jako object
df.loc[:,'SEX':'MARRIAGE']=df.loc[:,'SEX':'MARRIAGE'].astype('object')

In [5]:
#zamieniamy nazwe zmiennej objaśnianej na y

target='y'
df.rename(columns={'default payment next month':'y'},inplace=True)

In [6]:
#tworzymy listę zmiennych objaśniających

features=df.columns.tolist()
features.remove('y')

In [7]:
#zmieniam zmienna, teraz 1 oznacza klienta ktory splaci zobowiazanie, a 0 niewyplacalnego

df.y=1-df.y

In [8]:
#wpływ wieku może być nieliniowy, dodajmy więc kwadrat

df['AGE_SQ']=df.AGE.apply(lambda x: x**2)

In [9]:
#to samo zróbmy ze zmienną odpowiedzialną za zwklekanie ze spłatą, ze względu na występowanie zmiennych ujemnych dodajemy 2            

df.loc[:,"PAY_0":'PAY_6']+=2

for clmn in df.loc[:,'PAY_0':'PAY_6'].columns.tolist():
    df[f'{clmn}_SQ']=df[clmn].apply(lambda x: x**2)

In [10]:
#tworzymy zmienne zerojedynkowe odpowiadające kolejnym poziomom zmiennych dyskretnych

df2=pd.get_dummies(df,drop_first=True)

#dla algorytmów drzew losowych nie będzie potrzebne użycie label encodera dla naszego zbioru danych, ponieważ wszystkie zmienne są numeryczne

In [11]:
pandas_profiling.ProfileReport(df2).to_file('ML.html')

#zmienne BILL_AMT i MARRIAGE cechują się wysokim współczynnikiem korelacji, a zmienna PAY_AMT2 jest silnie skośna
#zmienne te mogą być problematyczne
#kwadraty zmiennych są naturalnie skorelowane

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
df2.y.value_counts()
#okolo 20% obserwacji z wartoscia 0, lepiej wiec uzyc Stratified KF

1    23364
0     6636
Name: y, dtype: int64

In [14]:
#zapisujemy df dla algorytmu drzew losowych i df2 dla pozostałych algorytmów

df.to_excel('credit_card_data.xlsx')

df2.to_excel('dumm_credit_card_data.xlsx')